• TAKE SAMPLES SHORTER THAN 20 SECONDS

• LOWERCASE ALL TRANSCRIPTS

• TURKISH TEXT NORMALIZATION (NORMALIZE NUMBERS, PUNCTIATIONS, ABBREVIATIONS, CLEAR SPECIAL CHARACTERS ETC.)

• SHUFFLE MANIFEST FILE

In [2]:
import json 
import re
import os
import math
import random 
from tqdm import tqdm

# Take samples shorter than 20 seconds and check filepaths are exists

In [15]:
file = "text_dataset.json"
file_out = "new_text_dataset.jsonl"

total_duration = 0

with open(file, "r", encoding="utf-8") as fin:
    with open(file_out, 'w', encoding="utf-8") as fout:
     
        for line in fin:
            original_file_json = json.loads(line)
              
            #CHANGE FILEPATH NAME
            original_file_json["audio_filepath"] = re.sub('/OLD/FILEPATH/', '/NEW/FILEPATH/', original_file_json["audio_filepath"])   
            
            # TAKE SAMPLES SHORTER THAN 20 SECONDS AND HIGHER THAN 1 SECOND.
            if original_file_json["duration"] < 20 and original_file_json["duration"] > 1:
                json.dump(original_file_json, fout, ensure_ascii=False)
                fout.write('\n')

                # SUM TOTAL TRAINING HOURS
                total_duration += original_file_json["duration"]
                
                # CHECK IF FILEPATH DIRECTORY IS EXISTS
                isFile = os.path.isfile(original_file_json["audio_filepath"]) 
                if isFile is False: # change False
                    print("This file does not exist:",original_file_json["audio_filepath"])
                                      
print("done!")

done!


# Total training data hours

In [16]:
total_duration/3600

261.1068836111104

# Turkish abbreviations and numbers

In [6]:
_abbreviations = [(re.compile('\\b%s\\.' % x[0], re.IGNORECASE), x[1]) for x in [
    ('adr', 'adres'),
    ('agy', 'adı geçen yapıt'),
    ('alb', 'albay'),
    ('ank', 'ankara'),
    ('ist', 'istanbul'),
    ('apt', 'apartman'),
    ('sok', 'sokak'),
    ('cad', 'cadde'),
    ('astsb', 'astsubay'),
    ('atgm', 'asteğmen'),
    ('bkz', 'bakınız'),
    ('bknz', 'bakınız'),
    ('bnb', 'binbaşı'),
    ('bşk', 'başkanlığı'),
    ('bştbp', 'baştabip'),
    ('bul', 'bulvarı'),
    ('bulv', 'bulvarı'),
    ('cal', 'kalori'),
    ('cm', 'santimetre'),
    ('m', 'metre'),
    ('gr', 'gram'),
    ('çvş', 'çavuş'),
    ('dl', 'desilitre'),
    ('dm', 'desimetre'),
    ('doç', 'doçent'),
    ('dr', 'doktor'),
    ('dz', 'deniz'),
    ('kuv', 'kuvvetleri'),
    ('yrb', 'yarbay'),
    ('yy', 'yüzyıl'),
    ('yard', 'yardımcı'),
    ('müh', 'mühendis'),
    ('ütğm', 'üsteğmen'),
    ('uzm', 'uzman'),
    ('müd', 'müdür'),
    ('mm', 'milimetre'),    
    ('mey', 'meydanı'),
    ('mim', 'mimar'),    
    ('mb', 'megabayt'),
    ('gb', 'cigabayt'),
    ('lt', 'litre'),
    ('ltd', 'limited'),
    ('kw', 'kilovat'),
    ('km', 'kilometre'),
    ('hrp', 'harp'),
    ('gen', 'general'),
    ('astsb', 'astsubay'),
    ('atgm', 'asteğmen'),
    ('ens', 'enstitüsü'),
    ('ecz', 'eczanesi'),
]]

single_digits = {
    0: "sıfır",
    1: "bir",
    2: "iki",
    3: "üç",
    4: "dört",
    5: "beş",
    6: "altı",
    7: "yedi",
    8: "sekiz",
    9: "dokuz"
}

double_digits = {
    1: "on",
    2: "yirmi",
    3: "otuz",
    4: "kırk",
    5: 'elli',
    6: 'altmış',
    7: 'yetmiş',
    8: 'seksen',
    9: 'doksan'
}

factorizations = [
    [1e2, "yüz"],
    [1e3, "bin"],
    [1e6, "milyon"],
    [1e9, "milyar"],
    [1e12, "trilyon"],
    [1e15, "katrilyon"],
    [1e18, "kentilyon"]
]

# Turkish text cleaning and normalization

### • Normalize urls

In [75]:
_url_re = re.compile(r'([a-zA-Z])\.(com|gov|org)')
def expand_urls(m):
    if m.group(2) == "com":
        group_2 = "kom"
    else:
        print(m.group(1))
        print(m.group(2))
        group_2 = m.group(2)
    return f'{m.group(1)} nokta {group_2}'

### • Normalize Turkish abbreviations

In [ ]:
def normalize_abbreviations(text):
    for regex, replacement in _abbreviations:
        text = re.sub(regex, replacement, text)
    return text

### • Clear punctiations and remove non-Turkish characters

In [3]:
def normalize_punctuations_and_special_characters(text):
    punc = '''!)"(,':;«»?�‘’'…“”'''
    
    if " gr " in text:
        text = text.replace(" gr ", " gram ")
    if " dk " in text:
        text = text.replace(" dk ", " dakika ")
    if " ml " in text:
        text = text.replace(" ml ", " mililitre ")
       
    for char in text:

        if char == "%":
            text = text.replace(char, "yüzde ")
        if char == "½":
            text = text.replace(char, "yarım") # CHECHK AUDIO FILE
        if char == "¼":
            text = text.replace(char, " çeyrek")
        if char == "°":
            text = text.replace(char, "derece")
        if char == "+":
            text = text.replace(char, " artı ")
        if char == "/":
            text = text.replace(char, " bölü ")
        if char == "*": 
            text = text.replace(char, " çarpı ")
      
        
        if char == "&":
            text = text.replace(char, "ve") # CHECHK AUDIO FILE
        if char == "�":
            text = text.replace(char, " ") # CHECHK AUDIO FILE

        if char in punc:
            text = text.replace(char, "")

        if "https":
            text = re.sub("https", "h t t p s", text)   
        if "http":
            text = re.sub("http", "h t t p", text)


        if char == "–":
            text = text.replace(char, " ")
        if char == "-":
            text = text.replace(char, " ")
        

        if char == "i̇" or char == "î":
            text = text.replace(char, "i")
        if char == "û":
            text = text.replace(char, "u")
        if char == "â":
            text = text.replace(char, "a")
        if char == "é":
            text = text.replace(char, "e")
        if char == "\xa0":
            text = text.replace(char, " ") 

    return text

### • Normalize numbers in Turkish

In [16]:
def convert_numbers(val):
    if val < 10:
        return single_digits[val]

    elif val < 100:
        normalized_number = double_digits[math.floor(val / 10)]
        if val % 10 == 0:
            return normalized_number
        return double_digits[math.floor(val / 10)] + " " + convert_numbers(val % 10)

    else:
        for idx, (div, factorization_name) in enumerate(factorizations):
            factorization_div = factorizations[idx + 1][0]

            if val < factorization_div:
                first_part = math.floor(val / div)
                second_part = val % div

                if first_part != 1:
                    normalized_number = convert_numbers(first_part) + " "
                else:
                    normalized_number = ""

                normalized_number += factorization_name

                if second_part != 0:
                    normalized_number += " " + convert_numbers(second_part)

                return normalized_number
            
def find_and_normalize_number(text):
    num = re.search("[0-9]+", text)
    find = num is not None

    if find:
        text = list(text)
        text[num.start():num.end()] = convert_numbers(int(num.group()))
        text = "".join(text)

    return find, text

def normalize_numbers(text):
    number_normalized, new_text = find_and_normalize_number(text)
    while number_normalized:
        number_normalized, new_text = find_and_normalize_number(new_text)

    return new_text

In [17]:
text = "naber nasılsın 5462456"
normalize_numbers(text)

'naber nasılsın beş milyon dört yüz altmış iki bin dört yüz elli altı'

# Create preprocessed manifest file

In [86]:
file_path = file_out
file_normalized = "normalized_manifest_file.jsonl"

original_vocab = set()
cleaned_vocab = set()

with open(file_path, 'r', encoding="utf-8") as f:
    with open(file_normalized, 'w', encoding="utf-8") as fp:
    
        for line in tqdm(liste):
            original_file_json = json.loads(line)
                
            original_vocab.update(list(original_file_json["text"]))
            
            text = original_file_json["text"]
            
            text = text.lower()
            text = re.sub(_url_re, expand_urls, text)
            text = normalize_numbers(text)
            text = normalize_abbreviations(text)
            new_text = normalize_punctuations_and_special_characters(text)

            cleaned_vocab.update(list(new_text))
            
            original_file_json["text"] = new_text
            json.dump(original_file_json, fp, ensure_ascii=False)
            fp.write('\n')
                    
print("original_vocab:", original_vocab) 
print("cleaned_vocab:", cleaned_vocab)

100%|████████████████████████████████████████████████████████████████████████| 119370/119370 [00:47<00:00, 2528.14it/s]

original_vocab: {':', '‘', 'ö', 'j', '̇', "'", '?', '�', '–', 'h', 'ı', '”', 'd', 'g', 's', '%', 't', 'x', ',', 'p', 'm', '½', '»', ' ', '’', 'ü', 'f', 'ğ', 'a', '…', 'v', 'r', '"', 'w', 'l', 'i', 'z', 'c', 'é', ';', '&', '«', '-', 'b', 'â', 'q', 'o', '!', '“', 'e', 'n', 'ş', '.', 'k', 'ç', 'y', 'u'}
cleaned_vocab: {'ö', 'j', '̇', '–', 'h', 'ı', 'd', 'g', 's', 't', 'x', 'p', 'm', ' ', 'ü', 'f', 'ğ', 'a', 'v', 'r', 'w', 'l', 'i', 'z', 'c', 'b', 'q', 'o', 'e', 'n', 'ş', '.', 'k', 'ç', 'y', 'u'}


# Shuffle the manifest file

In [87]:
liste = []

cleaned_file_path = file_normalized

with open(cleaned_file_path, 'r', encoding="utf-8") as f:  
    for line in f:
        liste.append(line) 
        
len(liste)

119370

In [88]:
random.shuffle(liste)
preprocessed_file_path = "preprocessed_manifest_file.jsonl"
with open(preprocessed_file_path, 'w', encoding="utf-8") as f:  
    for line in liste:
        f.write(line)
print("Manifest file ready for training!")

Manifest file ready for training!
